# SageMaker Tensorflow 컨테이너를 사용하여 하이퍼파라미터 튜닝하기
## [(원본)](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/hyperparameter_tuning/tensorflow_mnist)

이 문서는 **SageMaker TensorFlow container**를 사용하여 [MNIST dataset](http://yann.lecun.com/exdb/mnist/)을 훈련시키기 위해 convolutional neural network 모델을 만드는 방법에 초점을 두고 있습니다. 
이것은 하이퍼파라미터 튜닝을 활용하여 서로 다른 하이퍼파라미터를 조합하여 여러 훈련 Job을 실행함으로써 최상의 모델 훈련 결과를 제공하게 됩니다. 

## 환경 설정
워크플로우를 시작하기 전에 몇가지 설정이 필요합니다. 

1. 훈련 데이터셋과 모델 아티펙트가 저장될 s3버킷과 prefix를 지정합니다. 
2. SageMaker가 s3와 같은 리소스를 접근할 수 있도록 실행 Role을 가져옵니다. 

In [40]:
import sagemaker

bucket = '<My bucket name>'#sagemaker.Session().default_bucket() # we are using a default bucket here but you can change it to any bucket in your account
prefix = 'sagemaker/DEMO-hpo-tensorflow-high' # you can customize the prefix (subfolder) here

role = sagemaker.get_execution_role() # we are using the notebook instance role for training in this example

이제 필요한 Python 라이브러리를 import 합니다. 

In [29]:
import boto3
from time import gmtime, strftime
from sagemaker.tensorflow import TensorFlow
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

## MNIST 데이터셋 다운로드하기

In [30]:
import utils
from tensorflow.contrib.learn.python.learn.datasets import mnist
import tensorflow as tf

data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz
('Writing', 'data/train.tfrecords')
('Writing', 'data/validation.tfrecords')
('Writing', 'data/test.tfrecords')


## 데이터 업로드하기
 ```sagemaker.Session.upload_data``` 함수를 이용하여 S3경로에 데이터셋을 업로드합니다. 해당 함수의 리턴값은 S3의 경로를 가르킵니다. 이 경로는 훈련 Job을 시작할 때 사용할 것입니다. 

In [31]:
inputs = sagemaker.Session().upload_data(path='data', bucket=bucket, key_prefix=prefix+'/data/mnist')
print (inputs)

s3://sds-sm-seongshj/sagemaker/DEMO-hpo-tensorflow-high/data/mnist


## 분산 훈련을 위한 스크립트 작성하기
다음은 네트워크 모델의 전체코드입니다. 

In [32]:
!cat 'mnist.py'

import os
import tensorflow as tf
from tensorflow.python.estimator.model_fn import ModeKeys as Modes

INPUT_TENSOR_NAME = 'inputs'
SIGNATURE_NAME = 'predictions'

def model_fn(features, labels, mode, params):
    # this script takes learning_rate as a hyperparameter
    learning_rate = params.get("learning_rate",0.1)
    
    # Input Layer
    input_layer = tf.reshape(features[INPUT_TENSOR_NAME], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

   

이 스크립트는 [TensorFlow MNIST example](https://github.com/tensorflow/models/tree/master/official/mnist)를 수정한 것입니다. 
이것은 훈련, 평가, 추론을 위해 사용되는```model_fn(features, labels, mode)`` 를 제공합니다.

### 일반적인 ```model_fn```

일반적인 **```model_fn```** 은 다음과 같은 패턴을 따릅니다. 
1. [Neural network 정의](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L96)
- [Neural network에 ```features``` 적용](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L178)
- [```mode```가 ```PREDICT``` 이면 neural network에서 output 리턴](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L186)
- [Output과 ```labels```을 비교하는 loss fuction 계산](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L188)
- [Optimizer 생성 및 neural network 개선을 위한 loss function 최소화](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L193)
- [Output, optimizer, loss function 리턴](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L205)

### 분산 훈련에서의 ```model_fn``` 작성
분산 훈련이 일어나면, 동일한 neural network은 여러 훈련 인스턴스로 보내집니다. 개별 인스턴스는 데이터셋의 배치를 예측하고, loss를 계산하고 optimizer를 최소화합니다. 이 프로세스의 전체 루프를 **training step** 이라고 합니다. 



#### training steps 동기화
A [global step](https://www.tensorflow.org/api_docs/python/tf/train/global_step)은 인스턴스 사이에 공유되는 전역 변수입니다. 
그것은 분산 훈련에서 필요하기 때문에 Optimizer는 실행되는 중간에 **training steps** 의 수를 추척합니다. 


```python
train_op = optimizer.minimize(loss, tf.train.get_or_create_global_step())
```

이것이 분산훈련을 위해 필요한 유일한 변경입니다!

## 하이퍼파라미터 튜닝 Job 설정

*참고: 아래의 기본 설정에서는 하이퍼파라미터 튜닝 Job이 완료하는데 30분이 소요될 수 있습니다.*

이제 다음 단계에 따라 SageMaker Python SDK를 사용하여 하이퍼파라미터 튜닝 Job을 설정합니다.
* TensorFlow 훈련 Job 설정을 위한 estimator 생성하기
* 튜닝하려는 하이퍼파라미터 범위 정의하기. 이 예제에서는 "learning_rate"를 튜닝함 
* 최적화할 튜닝 Job의 목표 메트릭 정의하기
* 위의 설정과 튜닝 Resource Configuratin으로 하이퍼파라미터 Tuner 생성


SageMaker에서 단일 TensorFlow Job을 훈련하는 것과 유사하게, TensorFlow 스크립트, IAM role, (작업별)하드웨어 구성을 전달하는 TensorFlow estimator를 정의합니다. 

In [34]:
estimator = TensorFlow(entry_point='mnist.py',
                  role=role,
                  framework_version='1.12.0',
                  training_steps=1000, 
                  evaluation_steps=100,
                  train_instance_count=1,
                  train_instance_type='ml.m4.xlarge',
                  base_job_name='DEMO-hpo-tensorflow')

Once we've defined our estimator we can specify the hyperparameters we'd like to tune and their possible values.  We have three different types of hyperparameters.

estimator를 정의하고 나면 튜닝하려는 하이퍼파라미터과 가능한 값을 지정할 수 있습니다. 하이퍼파라미터는 세 가지 유형이 있습니다.

- 범주형 파라미터는 이산형 셋(discrete set)에서 하나의 값을 가져야 합니다. 가능한 값 목록을  `CategoricalParameter(list)`으로 전달하여 정의합니다 
- 연속형 파라미터는 `ContinuousParameter(min, max)` 에서 정의한 최소값과 최대값 사이의 실수 값을 가질 수 있습니다. 
- 정수형 파라미터는 `IntegerParameter(min, max)`에서 정의한 최소값과 최대값 사이의 정수 값을 가질 수 있습니다. 

 
*참고: 가능하면 값을 최소한의 restrictive type을 지정하는 것이 거의 항상 좋습니다. 예를 들면, learning rate는 연속값으로 0.01에서 0.2로 튜닝하는 것이 0.01, 0.1, 0.15 혹은 0.2의 범주형 파라미터로 튜닝하는 것보다 더 나은 결과를 얻을 수 있습니다.*

In [35]:
hyperparameter_ranges = {'learning_rate': ContinuousParameter(0.01, 0.02)}

다음으로 튜닝을 위한 목표 매트릭과 그것을 정의하기 위한 설정을 진행합니다. 이것은 훈련 Job이 CloudWatch 로그로부터 매트릭을 추출하는데 필요한 정규표현식(Regex)를 포함합니다. 이 경우, 스크립트는 loss값을 방출하고 목표 매트릭은 이를 목표 매트릭으로 사용할 것입니다. 또한 최상의 하이퍼파라미터 설정을 찾을 때, 목표 매트릭을 최소화하기 하이퍼파라미터를 튜닝하기 위해 objective_type은 'minimize'로 설정합니다. default로 objective_type은 'maximize' 설정됩니다. 



In [36]:
objective_metric_name = 'loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'loss',
                       'Regex': 'loss = ([0-9\\.]+)'}]


이제 `HyperparameterTuner` 객체를 생성할 것이고, 객체에 다음 값들이 전달됩니다.:
- 위에서 생성한 TensorFlow estimator
- 하이퍼파라미터 범위
- 목표 매트릭 이름 및 정의
- 총 훈련 Job의 갯수와 병렬적으로 실행할 훈련 Job의 수와 같은 튜닝 resource configurations 




In [37]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=9,
                            max_parallel_jobs=3,
                            objective_type=objective_type)


## 하이퍼파라미터 튜닝 Job 시작하기
마지막으로 `.fit()`을 호출하고 훈련 및 테스트 데이터셋의 S3 경로를 전달함에 따라 하이퍼파라미터 훈련 Job을 시작할 수 있습니다.

하이퍼파라미터 튜닝 Job이 생성된 후, 다음 단계에서 진행 상태를 보기위해 위해 튜닝 Job을 describe 할 수있어야 합니다. SageMaker의 콘솔->Jobs으로 이동하여 하이퍼파라미터의 튜닝 Job의 진행상태를 확인할 수 있습니다. 


In [38]:
tuner.fit(inputs)


하이퍼파라미터 튜닝 Job을 간단히 체크해하여 성공적으로 시작했는지 확인하시기 바랍니다. 


In [39]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

u'InProgress'

## 튜닝 Job 완료 후, 결과 분석하기
튜닝 Job 결과를 분석하기 위해 "HPO_Analyze_TuningJob_Results.ipynb"  예제를 참조하십시오.




## 최상의 모델 배포하기
이제 최상의 모델을 얻었으며, endpoint에서 배포할 수 있습니다. 모델을 배포하는 방법은 SageMaker sample notebook이나 SageMaker documentation을 참고하시기 바랍니다. 

